# Transformadas Wavelet y visualización de escalogramas

El siguiente archivo lleva a cabo diversas transformadas wavelet de los registros de la base de datos procesada. A partir de estas transformaciones, se generan los escalogramas correspondientes.

## Desarrollo

Se importan las bibliotecas necesarias para generar los escalogramas a partir de los registros de señales cerebrales.

In [3]:
import pandas as pd
import numpy as np
import pywt
import os
import math
import splitfolders

from matplotlib import pyplot as plt

Se carga la base de datos procesada.

In [16]:
df = pd.read_csv('BD_procesada/BD_procesada.csv', sep=';')
df.head(10)

,muestra 1,muestra 2,muestra 3,muestra 4,muestra 5,muestra 6,muestra 7,muestra 8,muestra 9,muestra 10,...,muestra 2492,muestra 2493,muestra 2494,muestra 2495,muestra 2496,muestra 2497,muestra 2498,muestra 2499,muestra 2500,estructura
0,-54,-14,-102,-286,-384,-379,-358,-340,-244,-144,...,45,100,58,31,1,-25,15,0,-4,1
1,-142,-242,-218,-144,-150,-214,-145,33,226,315,...,61,-59,-125,-86,70,162,234,189,151,1
2,2,7,15,27,47,45,-9,-75,-94,-71,...,-108,-81,-86,-27,-24,-11,-2,47,38,0
3,135,-96,-6,-100,10,-68,-138,-335,-373,-199,...,116,111,279,282,333,138,-17,-14,-200,0
4,-68,-91,67,119,83,128,-55,22,-139,-69,...,-18,17,157,255,322,467,188,298,199,0
5,70,8,74,-40,-38,70,50,31,-123,26,...,-6,9,-33,133,0,99,-91,19,-103,1
6,-29,-24,-26,-44,-20,15,-9,8,58,110,...,-227,-273,-267,-250,-221,-196,-157,-177,-188,0
7,218,108,281,-40,152,53,350,153,77,-260,...,29,-189,3,-419,-344,-506,-309,-430,-383,1
8,-56,-131,-113,20,108,100,-40,-78,-73,-16,...,13,-48,-104,-146,-159,-32,36,55,11,0
9,152,139,92,25,-8,14,95,145,156,174,...,-393,-405,-391,-330,-301,-289,-255,-181,-174,0


A continuación, se divide el dataframe principal en dos: uno que contiene los registros de las señales generadas por el núcleo subtalámico y otro que contiene las señales generadas por otras estructuras.

In [17]:
#Se crea un dataframe para las señales del núcleo subtalámico
df_stn = df[df['estructura'] == 1].copy()
df_stn.drop(columns=['estructura'], axis=1, inplace=True)
df_stn

,muestra 1,muestra 2,muestra 3,muestra 4,muestra 5,muestra 6,muestra 7,muestra 8,muestra 9,muestra 10,...,muestra 2491,muestra 2492,muestra 2493,muestra 2494,muestra 2495,muestra 2496,muestra 2497,muestra 2498,muestra 2499,muestra 2500
0,-54,-14,-102,-286,-384,-379,-358,-340,-244,-144,...,-74,45,100,58,31,1,-25,15,0,-4
1,-142,-242,-218,-144,-150,-214,-145,33,226,315,...,13,61,-59,-125,-86,70,162,234,189,151
5,70,8,74,-40,-38,70,50,31,-123,26,...,26,-6,9,-33,133,0,99,-91,19,-103
7,218,108,281,-40,152,53,350,153,77,-260,...,-133,29,-189,3,-419,-344,-506,-309,-430,-383
10,-293,-226,-155,-66,14,23,-19,-102,-105,-113,...,53,-64,-196,-200,-155,-110,-137,-101,-50,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,-196,-165,-235,-161,-100,-100,24,-90,33,-101,...,-61,-63,-57,35,-74,21,-119,-5,-135,69
3993,156,151,150,134,10,-44,-87,-139,-162,-176,...,-413,-221,-77,-49,-79,-105,-211,-255,-247,-146
3994,-355,-349,-383,-434,-315,-101,79,104,-71,-205,...,30,7,-115,-276,-357,-331,-202,-152,-200,-234
3996,134,167,178,186,85,40,63,39,-79,-144,...,82,112,155,165,170,133,80,-11,-109,-126


In [18]:
#Se crea un dataframe para las demás señales
df_otras = df[df['estructura'] == 0].copy()
df_otras.drop(columns=['estructura'], axis=1, inplace=True)
df_otras

,muestra 1,muestra 2,muestra 3,muestra 4,muestra 5,muestra 6,muestra 7,muestra 8,muestra 9,muestra 10,...,muestra 2491,muestra 2492,muestra 2493,muestra 2494,muestra 2495,muestra 2496,muestra 2497,muestra 2498,muestra 2499,muestra 2500
2,2,7,15,27,47,45,-9,-75,-94,-71,...,-88,-108,-81,-86,-27,-24,-11,-2,47,38
3,135,-96,-6,-100,10,-68,-138,-335,-373,-199,...,-95,116,111,279,282,333,138,-17,-14,-200
4,-68,-91,67,119,83,128,-55,22,-139,-69,...,-41,-18,17,157,255,322,467,188,298,199
6,-29,-24,-26,-44,-20,15,-9,8,58,110,...,-149,-227,-273,-267,-250,-221,-196,-157,-177,-188
8,-56,-131,-113,20,108,100,-40,-78,-73,-16,...,50,13,-48,-104,-146,-159,-32,36,55,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989,-48,-129,-190,-239,-235,-186,-171,-118,-106,-39,...,82,84,207,407,442,314,289,301,303,304
3991,-202,-270,-247,-247,-244,-286,-297,-269,-268,-230,...,238,257,199,144,97,90,116,116,89,110
3995,36,6,37,122,201,257,242,192,106,62,...,145,195,291,259,186,81,117,108,106,56
3998,-489,-471,-174,-189,-133,-73,-161,-78,-377,-216,...,-157,17,-32,84,152,-56,-71,-310,-24,-94


A continuación se listan las transformadas wavelet continuas que se utilizarán en el desarrollo del proyecto.

In [4]:
wavlist = ['cgau1', 'cmor', 'mexh']

A continuación, se definen una serie de funciones que ayudan a realizar las transformaciones requeridas y a guardar los resultados como escalogramas.

In [20]:
def aplicar_cwt_y_guardar_imagen(
        nombre_trans:str,
        registro:np.ndarray,
        ruta:str,
        m:int
):
    """
    Aplica la transformada continua wavelet (CWT) a una señal, generando el 
    escalograma correspondiente y luego guarda la imagen resultante en una 
    ubicación específica.

    Parámetros:
        nombre_trans (str): Nombre de la transformada de wavelet a aplicar.
        registro (np.ndarray): Array que contiene el registro de la señal.
        ruta (str): Ruta donde se guardará la imagen resultante.
        m (int): Índice utilizado para nombrar la imagen resultante.

    Devoluciones:
        None

    Ejemplo:
        # Aplicar la transformada wavelet a una señal en un array y guardar la imagen resultante
        aplicar_cwt_y_guardar_imagen(nombre_trans='mexh', registro=array, ruta='imagenes', m=0)

    Lanza:
        TypeError: Si los tipos de los parámetros no son correctos.
        ValueError: Si la ruta especificada no es válida.
    """
        
    nombre_imagen = os.path.join(ruta, f"Figura_{m}.png")
    
    # Escalas para la CWT
    scales = np.arange(1, 100)

    # Realizar la transformada continua de wavelet (CWT) 
    mexh_cwt, frequencies = pywt.cwt(registro, scales, nombre_trans)

    # Plotear el resultado
    plt.figure(figsize=(2.5,2.5))
    plt.imshow(np.abs(mexh_cwt), extent=[0, 0.1, scales[-1], scales[0]], aspect='auto', cmap='jet')
    plt.axis('off')
    plt.gca().set_position([0,0,1,1])
    plt.savefig(nombre_imagen, bbox_inches='tight', pad_inches=0)
    plt.close()

A continuación, se procede a aplicar las diferentes transformadas especificadas en el array wavlist a las señales almacenadas en los dataframes df_stn y df_otras.

In [21]:
for transformada in wavlist:
    #Se genera las transformadas al conjunto de datos df_stn
    ruta_imagenes_stn = os.path.join("imagenes", transformada, "stn")    
    for i, registro in enumerate(df_stn.values):
        # Se itera sobre cada fila del DataFrame 'df_stn' junto con su índice,
        # utilizando 'enumerate' para obtener tanto el índice 'i' como el registro directamente
        aplicar_cwt_y_guardar_imagen(
            nombre_trans = transformada,
            registro = registro,
            ruta = ruta_imagenes_stn,
            m=i
        )

    #Se genera las transformadas al conjunto de datos df_otras
    ruta_imagenes_otras = os.path.join("imagenes", transformada, "otras")    
    for i, registro in enumerate(df_otras.values):
        # Se itera sobre cada fila del DataFrame 'df_otras' junto con su índice,
        # utilizando 'enumerate' para obtener tanto el índice 'i' como el registro directamente
        aplicar_cwt_y_guardar_imagen(
            nombre_trans = transformada,
            registro = registro,
            ruta = ruta_imagenes_otras,
            m=i
        )
        

Se procede a realizar un split a nivel de directorios.

In [5]:
for transformada in wavlist:
    # Ruta al directorio de origen que contiene las subcarpetas con imágenes de cada clase
    input_folder = "imagenes/{}/".format(transformada)
    # Ruta al directorio de salida donde se guardarán las carpetas divididas
    output_foler = "split_imagenes/{}/".format(transformada)
    # Dividir las imágenes en carpetas de entrenamiento, validación y prueba con una proporción de 80%/10%/10%
    splitfolders.ratio(input=input_folder, output=output_foler, seed=1337, ratio=(.8,.1,.1))

Copying files: 4000 files [00:11, 353.56 files/s]
Copying files: 4000 files [00:11, 349.93 files/s]
Copying files: 4000 files [00:11, 336.57 files/s]
